In [ ]:
from analysis import analysis_utils
import re
import pandas as pd

In [ ]:


metric_name = "overall_acc"

df = analysis_utils.wandb_to_df(
	[
		# "exp_14_stacking",
		# "exp_14_stacking_rtol0.01",
		# "exp_11",
		# "exp_11_boosting",
		# # "exp_12_rtol",
		# "exp_13_rol_boosting",
		# "exp_004_clean"
		# "exp_011_rtol0.01",
		# "exp_011_rtol0.001",
		# "exp_011_autoencoders",
		# "exp_011_autoencoders_50",
		# "exp_012_rtol0.01",
		# "exp_013_12foiautoencoders_rtol0.01",
		# "exp_014_rtol_defato_0.01",
		"exp0007",
		"exp0009_stack_hidden_maxlayers2_noappend",
		"exp0009_maxlayers1",
		"exp0009_maxlayers2",
		"exp0009_stack_hidden_maxlayers2",
		"exp0016",
		"exp0016_tanh",
		"exp0016_relu",
	],
	metric_name,
)
df = df.sort_index(axis=1)

In [ ]:
df.head()

In [ ]:

metric_columns = [
    c
    for c in df.columns
    if re.match(r"test_.*[1\-nn|3\-nn|svm|xgboost|rf|autoconstructive]_*" + metric_name, c)
    # if re.match("test_.*[drl]_" + metric, c)
]

df_filtered_tmp = df[metric_columns + ["project", "dataset_name"]]
df_filtered = pd.DataFrame()
for project in df_filtered_tmp['project'].unique():
    df_p = df[df['project'] == project]
    for dataset_name in df_p['dataset_name'].unique():
        df_p_d = df_p[df_p['dataset_name'] == dataset_name]
        tmp_df = df_p_d.iloc[:20, :]

        df_filtered = pd.concat((df_filtered, tmp_df))

df_wilcoxon = analysis_utils.wilcoxon_tests(df_filtered, metric_name)
df_wilcoxon.to_csv("analysis_wilcoxon.csv")

df_pivot = df_wilcoxon[df_wilcoxon["g1"] == f"test_drl_untrained_{metric_name}"].pivot(
    index=["project", "dataset_name"],
    columns="g2",
    values=["g1_mean", "g2_mean", "wilcoxon_result"],
)
df_pivot.columns = df_pivot.columns.swaplevel(0, 1)
df_pivot.sort_index(1).to_csv("pivot_untrained.csv")

df_pivot = df_wilcoxon[df_wilcoxon["g1"] == f"test_drl_{metric_name}"].pivot(
    index=["project", "dataset_name"],
    columns="g2",
    values=["g1_mean", "g2_mean", "wilcoxon_result"],
)
df_pivot.columns = df_pivot.columns.swaplevel(0, 1)
df_pivot.sort_index(1).to_csv("pivot_trained.csv")

# plot_html(df)
# df[["project", "dataset_name", "g2", "wilcoxon_result"]].pivot(
#     "project", "dataset_name", "g2", "wilcoxon_result"
# ).to_csv("pivot.csv")
df_filtered.melt(["project", "dataset_name"]).groupby(
    ["project", "dataset_name", "variable"]
).mean().unstack([0, 2]).to_csv("analysis2.csv")
avg = df_wilcoxon.groupby(["project", "dataset_name"]).mean()
avg.to_csv("analysis.csv")
with open("analysis.html", "w") as html_file:
    html_file.write(avg.style.highlight_max(color="lightgreen", axis=1).render())
print(df_wilcoxon)